In [ ]:
import os
from pathlib import Path

import pandas as pd
import requests
from sleeper_wrapper import Players

from draft_optimizer.src.utils import DATA_DIR

# Get Players

In [ ]:
# Get players
players_raw = Players().get_all_players()
players_df = pd.DataFrame.from_dict(players_raw).T

In [ ]:
# Subset and rename columns
info_cols = ["full_name", "active", "team"]
pos_cols = ["position", "fantasy_positions"]
id_cols = [c for c in players_df.columns if "id" in c]
keep_cols = info_cols + pos_cols + id_cols
players = players_df[keep_cols].rename({"player_id": "sleeper_id"}, axis=1).set_index("sleeper_id")

# Fix defenses
is_def = players["position"] == "DEF"
players.loc[is_def, "position"] = "D/ST"
players.loc[is_def, "fantasy_positions"] = "[D/ST]"
players.loc[is_def, "full_name"] = players_df.loc[is_def, "last_name"] + " D/ST"
# TODO: other league IDs?

print(players.shape)
players.head()

In [ ]:
# Output
out_dir = os.path.join(DATA_DIR, "sleeper")
if not os.path.isdir(out_dir):
    Path(out_dir).mkdir(parents=True, exist_ok=True)
players.to_csv(os.path.join(out_dir, "players.csv"), index=True)

# Get Data

In [ ]:
# Specify URL
weekly_url = "https://api.sleeper.com/{mode}/nfl/{year}/{week}?season_type=regular&position[]=DEF&position[]=K&position[]=QB&position[]=RB&position[]=TE&position[]=WR"

In [ ]:
def get_weekly_data(mode: str, year: int):
    # Loop over weeks
    weeks = list(range(1, 18 + 1))  # 1-18
    dfs = []
    for week in weeks:
        # Get weekly data
        raw = requests.get(weekly_url.format(mode=mode, year=year, week=week)).json()
        raw = [
            {"player_id": v["player_id"], "week": v["week"], "opponent": v["opponent"], **v["stats"]}
            for v in raw
            if v["game_id"] is not None
        ]
        if len(raw) == 0:
            continue
        df = pd.DataFrame(raw).set_index("player_id")
        df.index.name = "sleeper_id"

        # Append
        dfs.append(df)

    if len(dfs) > 0:
        # Concat projections
        out = pd.concat(dfs, axis=0)

        # Output
        out_dir = os.path.join(DATA_DIR, "sleeper", str(year))
        if not os.path.isdir(out_dir):
            Path(out_dir).mkdir(parents=True, exist_ok=True)
        out.to_csv(os.path.join(out_dir, f"{mode}.csv"), index=True)

In [ ]:
# Loop over years
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
for year in years:
    get_weekly_data("stats", year)
    get_weekly_data("projections", year)